In [1]:
import os

import nibabel as nib
import numpy as np
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import load_tractogram, save_tractogram
from dipy.io.utils import (create_nifti_header, get_reference_info,
                           is_header_compatible)
from dipy.tracking.streamline import select_random_set_of_streamlines
from dipy.tracking.utils import density_map

from dipy.data.fetcher import (fetch_file_formats,
                               get_file_formats)
from dipy.io.image import load_nifti_data, load_nifti, save_nifti
from dipy.io.streamline import save_trk, save_vtk, load_vtk


In [17]:
# Enables/disables interactive visualization
cwd = os.getcwd()
bundles_filename = cwd + '\\ThalamicRadiationL.trk'
ref_anat_filename = cwd + '\\white.nii.gz'

reference_anatomy = nib.load(ref_anat_filename)

In [18]:
affine, dimensions, voxel_sizes, voxel_order = get_reference_info(
    ref_anat_filename)
print(affine)
print(dimensions)
print(voxel_sizes)
print(voxel_order)

[[  -0.7    0.     0.    90. ]
 [   0.     0.7    0.  -126. ]
 [   0.     0.     0.7  -72. ]
 [   0.     0.     0.     1. ]]
[260 311 260]
[0.7 0.7 0.7]
LAS


Load tractogram will support 5 file formats, functions like load_trk or
load_tck will simply be restricted to one file format

TRK files contain their own header (when written properly), so they
technically do not need a reference. (See how below)

``cc_trk = load_tractogram(bundles_filename[0], 'same')``



In [19]:
cc_trk = load_tractogram(bundles_filename,'same')
#lpt_sft = load_tractogram(bundles_filename, reference_anatomy,
#                          bbox_valid_check=False)

In [20]:
affine, dimensions, voxel_sizes, voxel_order = get_reference_info(
    bundles_filename)
print(affine)
print(dimensions)
print(voxel_sizes)
print(voxel_order)

[[ -0.7  -0.    0.   90. ]
 [  0.   -0.7   0.   91. ]
 [  0.   -0.    0.7 -72. ]
 [  0.    0.    0.    1. ]]
[260 311 260]
[0.7 0.7 0.7]
LPS


In [21]:
#sft = StatefulTractogram(cc_trk, img, Space.VOX)
#bbox_valid_check =False
#sft.remove_invalid_streamlines()
save_vtk(cc_trk, "ThalamicRadiationL.vtk")

Once loaded, no matter the original file format, the stateful tractogram is
self-contained and maintains a valid state. By requiring a reference the
tractogram's spatial transformation can be easily manipulated.

Let's save all files as TRK to visualize in TrackVis for example.
However, when loaded the lpt and rpt files contain invalid streamlines and
for particular operations/tools/functions it is safer to remove them



Some functions in DIPY require streamlines to be in voxel space so
computation can be performed on a grid (connectivity matrix, ROIs masking,
density map). The stateful tractogram class provides safe functions for such
manipulation. These functions can be called safely over and over, by knowing
in which state the tractogram is operating, and compute only necessary
transformations

No matter the state, functions such as ``save_tractogram`` or
``removing_invalid_coordinates`` can be called safely and the transformations
are handled internally when needed.

